<a href="https://colab.research.google.com/github/psygrammer/psypy_lm/blob/main/notebooks/ch01/ch01_start_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Getting Started with the Model Architecture of the Transformer